In [1]:
import sqlite3
import duckdb
import pandas as pd

### Conexão e Teste

In [2]:
sqlite_con = sqlite3.connect('data/northwind.db')

df_customers = pd.read_sql_query("SELECT * FROM Customers LIMIT 5", sqlite_con)

sqlite_con.close()

print("\nExemplo de dados da tabela 'Customers':")

print(df_customers)


Exemplo de dados da tabela 'Customers':
  CustomerID                         CompanyName         ContactName  \
0      ALFKI                 Alfreds Futterkiste        Maria Anders   
1      ANATR  Ana Trujillo Emparedados y helados        Ana Trujillo   
2      ANTON             Antonio Moreno Taquería      Antonio Moreno   
3      AROUT                     Around the Horn        Thomas Hardy   
4      BERGS                  Berglunds snabbköp  Christina Berglund   

           ContactTitle                        Address         City  \
0  Sales Representative                  Obere Str. 57       Berlin   
1                 Owner  Avda. de la Constitución 2222  México D.F.   
2                 Owner                Mataderos  2312  México D.F.   
3  Sales Representative                120 Hanover Sq.       London   
4   Order Administrator                Berguvsvägen  8        Luleå   

            Region PostalCode  Country           Phone             Fax  
0   Western Europe      12

### Criar Tabelas Duckdb em memória

In [3]:
duckdb_con = duckdb.connect(database=':memory:')

duckdb_con.execute("CREATE TABLE Customers AS SELECT * FROM sqlite_scan('data/northwind.db', 'Customers')")
duckdb_con.execute("CREATE TABLE Employees AS SELECT * FROM sqlite_scan('data/northwind.db', 'Employees')")
duckdb_con.execute("CREATE TABLE Orders AS SELECT * FROM sqlite_scan('data/northwind.db', 'Orders')")
duckdb_con.execute("CREATE TABLE OrderDetails AS SELECT * FROM sqlite_scan('data/northwind.db', 'Order Details')")
duckdb_con.execute("CREATE TABLE Products AS SELECT * FROM sqlite_scan('data/northwind.db', 'Products')")
duckdb_con.execute("CREATE TABLE Categories AS SELECT * FROM sqlite_scan('data/northwind.db', 'Categories')")

### Análises

In [4]:
# 1. Contagem de clientes por país
conta_clientes = duckdb_con.sql("""
    SELECT Country, COUNT(*) AS CustomerCount
    FROM Customers
    GROUP BY Country
    ORDER BY CustomerCount DESC
""")

print("1. Contagem de clientes por país:")

print(conta_clientes)

1. Contagem de clientes por país:
┌─────────────┬───────────────┐
│   Country   │ CustomerCount │
│   varchar   │     int64     │
├─────────────┼───────────────┤
│ USA         │            13 │
│ Germany     │            11 │
│ France      │            11 │
│ Brazil      │             9 │
│ UK          │             7 │
│ Spain       │             5 │
│ Mexico      │             5 │
│ Venezuela   │             4 │
│ Italy       │             3 │
│ Canada      │             3 │
│ Argentina   │             3 │
│ Portugal    │             2 │
│ Sweden      │             2 │
│ Austria     │             2 │
│ Belgium     │             2 │
│ Denmark     │             2 │
│ Switzerland │             2 │
│ NULL        │             2 │
│ Finland     │             2 │
│ Ireland     │             1 │
│ Norway      │             1 │
│ Poland      │             1 │
├─────────────┴───────────────┤
│ 22 rows           2 columns │
└─────────────────────────────┘



In [5]:
# 2. Contagem de funcionários por Cargo
funcionario_titulo = duckdb_con.sql("""
    SELECT Title, COUNT(*) AS EmployeeCount
    FROM Employees
    GROUP BY Title
    ORDER BY EmployeeCount DESC
""")

print("\n2. Contagem de funcionários por título:")

print(funcionario_titulo)


2. Contagem de funcionários por título:
┌──────────────────────────┬───────────────┐
│          Title           │ EmployeeCount │
│         varchar          │     int64     │
├──────────────────────────┼───────────────┤
│ Sales Representative     │             6 │
│ Sales Manager            │             1 │
│ Inside Sales Coordinator │             1 │
│ Vice President, Sales    │             1 │
└──────────────────────────┴───────────────┘



In [6]:
# 3. Receita total por cliente
receita_total_cliente = duckdb_con.sql("""
    SELECT C.CustomerID, C.CompanyName, SUM(OD.UnitPrice * OD.Quantity * (1 - OD.Discount)) AS TotalRevenue
    FROM Orders O
    JOIN OrderDetails OD ON O.OrderID = OD.OrderID
    JOIN Customers C ON O.CustomerID = C.CustomerID
    GROUP BY C.CustomerID, C.CompanyName
    ORDER BY TotalRevenue DESC
""")

print("\n3. Receita total por cliente:")

print(receita_total_cliente)


3. Receita total por cliente:
┌────────────┬────────────────────────────────────┬────────────────────┐
│ CustomerID │            CompanyName             │    TotalRevenue    │
│  varchar   │              varchar               │       double       │
├────────────┼────────────────────────────────────┼────────────────────┤
│ BSBEV      │ B's Beverages                      │  6154115.340000002 │
│ HUNGC      │ Hungry Coyote Import Store         │  5698023.669999998 │
│ RANCH      │ Rancho grande                      │  5559110.080000002 │
│ GOURL      │ Gourmet Lanchonetes                │        5552309.805 │
│ ANATR      │ Ana Trujillo Emparedados y helados │  5534356.650000001 │
│ RICAR      │ Ricardo Adocicados                 │  5524517.309999998 │
│ FOLIG      │ Folies gourmandes                  │  5505502.849999996 │
│ LETSS      │ Let's Stop N Shop                  │  5462198.022499999 │
│ LILAS      │ LILA-Supermercado                  │         5437438.34 │
│ PRINI      │ Princ

In [7]:
# 4. Receita média por categoria de produto
receita_media_cat  = duckdb_con.sql("""
    SELECT 
        C.CategoryName, 
        AVG(OD.UnitPrice * OD.Quantity * (1 - OD.Discount)) AS AvgRevenuePerCategory
    FROM OrderDetails OD
    JOIN Products P ON OD.ProductID = P.ProductID
    JOIN Categories C ON P.CategoryID = C.CategoryID
    GROUP BY C.CategoryName
    ORDER BY AvgRevenuePerCategory DESC
""")

print("\n4. Receita média por categoria de produto:")

print(receita_media_cat )


4. Receita média por categoria de produto:
┌────────────────┬───────────────────────┐
│  CategoryName  │ AvgRevenuePerCategory │
│    varchar     │        double         │
├────────────────┼───────────────────────┤
│ Meat/Poultry   │    1370.6802148410268 │
│ Beverages      │      969.291925791151 │
│ Produce        │     826.8716466066554 │
│ Dairy Products │     732.2468767432764 │
│ Confections    │      645.409821227039 │
│ Condiments     │     587.3913208510555 │
│ Seafood        │     525.9777917175913 │
│ Grains/Cereals │     515.6774429151624 │
└────────────────┴───────────────────────┘



In [8]:
# 5. Número de pedidos por funcionário ao longo do tempo
pedido_func_tempo = duckdb_con.sql("""
    SELECT 
        CONCAT(e.FirstName, ' ', e.LastName) AS EmployeeName, 
        o.OrderDate, 
        COUNT(o.OrderID) OVER (PARTITION BY o.EmployeeID ORDER BY o.OrderDate) AS CumulativeOrders
    FROM Orders o
    JOIN Employees e ON o.EmployeeID = e.EmployeeID
""")

print("\n5. Número de pedidos por funcionário ao longo do tempo (cumulativo):")

print(pedido_func_tempo)


5. Número de pedidos por funcionário ao longo do tempo (cumulativo):
┌─────────────────┬─────────────────────┬──────────────────┐
│  EmployeeName   │      OrderDate      │ CumulativeOrders │
│     varchar     │      timestamp      │      int64       │
├─────────────────┼─────────────────────┼──────────────────┤
│ Steven Buchanan │ 2012-07-11 21:17:46 │                1 │
│ Steven Buchanan │ 2012-07-16 09:41:10 │                2 │
│ Steven Buchanan │ 2012-07-19 13:35:20 │                3 │
│ Steven Buchanan │ 2012-07-21 02:53:38 │                4 │
│ Steven Buchanan │ 2012-07-23 10:41:56 │                5 │
│ Steven Buchanan │ 2012-07-24 17:00:06 │                6 │
│ Steven Buchanan │ 2012-07-30 01:48:16 │                7 │
│ Steven Buchanan │ 2012-07-31 05:55:45 │                8 │
│ Steven Buchanan │ 2012-08-01 00:41:11 │                9 │
│ Steven Buchanan │ 2012-08-04 02:37:33 │               10 │
│       ·         │          ·          │                · │
│       ·      

In [9]:
# 6. Clientes com mais de 10 pedidos
cliente_10 = duckdb_con.sql("""
    WITH CustomerOrders AS (
        SELECT CustomerID, COUNT(OrderID) AS OrderCount
        FROM Orders
        GROUP BY CustomerID
    )
    SELECT C.CustomerID, C.CompanyName, CO.OrderCount
    FROM Customers C
    JOIN CustomerOrders CO ON C.CustomerID = CO.CustomerID
    WHERE CO.OrderCount > 10
""")

print("\n6. Clientes com mais de 10 pedidos:")

print(cliente_10)


6. Clientes com mais de 10 pedidos:
┌────────────┬────────────────────────────────────┬────────────┐
│ CustomerID │            CompanyName             │ OrderCount │
│  varchar   │              varchar               │   int64    │
├────────────┼────────────────────────────────────┼────────────┤
│ ALFKI      │ Alfreds Futterkiste                │        163 │
│ ANATR      │ Ana Trujillo Emparedados y helados │        195 │
│ ANTON      │ Antonio Moreno Taquería            │        163 │
│ AROUT      │ Around the Horn                    │        156 │
│ BERGS      │ Berglunds snabbköp                 │        178 │
│ BLAUS      │ Blauer See Delikatessen            │        164 │
│ BLONP      │ Blondesddsl père et fils           │        165 │
│ BOLID      │ Bólido Comidas preparadas          │        182 │
│ BONAP      │ Bon app'                           │        167 │
│ BOTTM      │ Bottom-Dollar Markets              │        185 │
│   ·        │           ·                        │  

In [10]:
# 7. Receita total e média por mês
receita_total_media = duckdb_con.sql("""
    SELECT 
        strftime('%Y-%m', CAST(OrderDate AS DATE)) AS OrderMonth, 
        SUM(UnitPrice * Quantity * (1 - Discount)) AS TotalRevenue,
        AVG(SUM(UnitPrice * Quantity * (1 - Discount))) OVER (ORDER BY strftime('%Y-%m', CAST(OrderDate AS DATE))) AS AvgRevenue
    FROM OrderDetails
    JOIN Orders ON OrderDetails.OrderID = Orders.OrderID
    GROUP BY OrderMonth
    ORDER BY OrderMonth
""")

print("\n7. Receita total e média por mês:")

print(receita_total_media)


7. Receita total e média por mês:
┌────────────┬────────────────────┬────────────────────┐
│ OrderMonth │    TotalRevenue    │     AvgRevenue     │
│  varchar   │       double       │       double       │
├────────────┼────────────────────┼────────────────────┤
│ 2012-07    │ 2066219.4000000004 │ 2066219.4000000004 │
│ 2012-08    │         3556875.79 │        2811547.595 │
│ 2012-09    │ 3440144.9800000004 │  3021080.056666667 │
│ 2012-10    │ 3201529.9599999995 │       3066192.5325 │
│ 2012-11    │ 2980494.7399999998 │ 3049052.9740000004 │
│ 2012-12    │ 3577936.8500000006 │  3137200.286666667 │
│ 2013-01    │         3075418.29 │ 3128374.2871428574 │
│ 2013-02    │ 2964192.8599999994 │      3107851.60875 │
│ 2013-03    │ 3471361.2099999995 │ 3148241.5644444446 │
│ 2013-04    │ 3262893.5199999996 │ 3159706.7600000002 │
│    ·       │              ·     │          ·         │
│    ·       │              ·     │          ·         │
│    ·       │              ·     │          ·       

In [11]:
# 8. Melhor mês de vendas para cada funcionário
melhor_mes_func = duckdb_con.sql("""
    WITH MonthlySales AS (
        SELECT 
            o.EmployeeID, 
            strftime('%Y-%m', CAST(o.OrderDate AS DATE)) AS OrderMonth, 
            SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) AS MonthlyRevenue
        FROM Orders o
        JOIN OrderDetails od ON o.OrderID = od.OrderID
        GROUP BY o.EmployeeID, OrderMonth
    ),
    RankedSales AS (
        SELECT 
            e.EmployeeID,
            CONCAT(e.FirstName, ' ', e.LastName) AS EmployeeName, 
            ms.OrderMonth, 
            ms.MonthlyRevenue,
            RANK() OVER (PARTITION BY e.EmployeeID ORDER BY ms.MonthlyRevenue DESC) AS Rank
        FROM MonthlySales ms
        JOIN Employees e ON ms.EmployeeID = e.EmployeeID
    )
    SELECT EmployeeName, OrderMonth, MonthlyRevenue
    FROM RankedSales
    WHERE Rank = 1
""")

print("\n8. Melhor mês de vendas para cada funcionário:")

print(melhor_mes_func)


8. Melhor mês de vendas para cada funcionário:
┌──────────────────┬────────────┬───────────────────┐
│   EmployeeName   │ OrderMonth │  MonthlyRevenue   │
│     varchar      │  varchar   │      double       │
├──────────────────┼────────────┼───────────────────┤
│ Laura Callahan   │ 2016-05    │         867720.25 │
│ Steven Buchanan  │ 2023-07    │ 726352.1699999999 │
│ Anne Dodsworth   │ 2013-07    │ 686016.5300000001 │
│ Andrew Fuller    │ 2017-11    │         627003.55 │
│ Nancy Davolio    │ 2014-05    │         626065.04 │
│ Robert King      │ 2020-08    │ 730154.9199999999 │
│ Margaret Peacock │ 2020-09    │ 701766.1100000001 │
│ Janet Leverling  │ 2013-12    │ 755028.3200000001 │
│ Michael Suyama   │ 2015-10    │ 641155.5099999999 │
└──────────────────┴────────────┴───────────────────┘



### Fechar Conexão SQLite

In [12]:
sqlite_con.close()

### Persistir Consultas no Duckdb (DW)

In [13]:
duckdb_con = duckdb.connect('dw.duckdb')

conta_clientes_df = conta_clientes.df()
duckdb_con.register('conta_clientes_df', conta_clientes_df)
duckdb_con.execute("CREATE TABLE conta_clientes AS SELECT * FROM conta_clientes_df")

funcionario_titulo_df = funcionario_titulo.df()
duckdb_con.register('funcionario_titulo_df', funcionario_titulo_df)
duckdb_con.execute("CREATE TABLE funcionario_titulo AS SELECT * FROM funcionario_titulo_df")

receita_total_cliente_df = receita_total_cliente.df()
duckdb_con.register('receita_total_cliente_df', receita_total_cliente_df)
duckdb_con.execute("CREATE TABLE receita_total_cliente AS SELECT * FROM receita_total_cliente_df")

receita_media_cat_df = receita_media_cat.df()
duckdb_con.register('receita_media_cat_df', receita_media_cat_df)
duckdb_con.execute("CREATE TABLE receita_media_cat AS SELECT * FROM receita_media_cat_df")

pedido_func_tempo_df = pedido_func_tempo.df()
duckdb_con.register('pedido_func_tempo_df', pedido_func_tempo_df)
duckdb_con.execute("CREATE TABLE pedido_func_tempo AS SELECT * FROM pedido_func_tempo_df")

cliente_10_df = cliente_10.df()
duckdb_con.register('cliente_10_df', cliente_10_df)
duckdb_con.execute("CREATE TABLE cliente_10 AS SELECT * FROM cliente_10_df")

receita_total_media_df = receita_total_media.df()
duckdb_con.register('receita_total_media_df', receita_total_media_df)
duckdb_con.execute("CREATE TABLE receita_total_media AS SELECT * FROM receita_total_media_df")

melhor_mes_func_df = melhor_mes_func.df()
duckdb_con.register('melhor_mes_func_df', melhor_mes_func_df)
duckdb_con.execute("CREATE TABLE melhor_mes_func AS SELECT * FROM melhor_mes_func_df")

# Fechar a conexão
duckdb_con.close()

In [17]:
duckdb_con = duckdb.connect('dw.duckdb')

select_ = duckdb_con.sql("SELECT * FROM conta_clientes")

print(select_)

duckdb_con.close()

┌─────────────┬───────────────┐
│   Country   │ CustomerCount │
│   varchar   │     int64     │
├─────────────┼───────────────┤
│ USA         │            13 │
│ France      │            11 │
│ Germany     │            11 │
│ Brazil      │             9 │
│ UK          │             7 │
│ Mexico      │             5 │
│ Spain       │             5 │
│ Venezuela   │             4 │
│ Canada      │             3 │
│ Argentina   │             3 │
│ Italy       │             3 │
│ Denmark     │             2 │
│ Finland     │             2 │
│ Portugal    │             2 │
│ Sweden      │             2 │
│ Austria     │             2 │
│ Belgium     │             2 │
│ NULL        │             2 │
│ Switzerland │             2 │
│ Poland      │             1 │
│ Ireland     │             1 │
│ Norway      │             1 │
├─────────────┴───────────────┤
│ 22 rows           2 columns │
└─────────────────────────────┘

